# Langchain 1.0

- langchain 1.0 실습 가이드입니다.
- 0버전과 문법 및 객체 구조가 매우 다르므로 GPT를 통한 코드 작성이 매우 어렵습니다. 반드시 작동이 확인된 예시 코드와 공식 문서를 기반으로 개발하시기 바랍니다.
  - https://docs.langchain.com/oss/python/langchain/quickstart
  - https://reference.langchain.com/python/langgraph/graphs/
- markdown셀의 header로 목차를 확인하고 코드 흐름을 따라가면서 이해되지 않는 개념이 있으면 함께 드린 마크다운 파일을 다시 읽어보세요!

## 0. 환경 설정

- Python 3.12.12 기반의 conda 가상 환경에서 테스트되었습니다.
- environment.yml 또는 requirements.txt 파일을 통해 환경을 재현하실 수 있습니다.
- 환경 변수만 잘 등록해준다면 venv, colab 등 다른 실행 환경에서도 작동할 것으로 예상되나 버전 정보를 잘 확인해주세요.

In [1]:
# 공통 라이브러리 import
import os

# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

# 예쁜 출력
from pprint import pprint

## 1. langsmith 연결
- langsmith는 langgraph|langchain Agent의 실행을 기록하고, 시각화 및 분석을 제공하는 Observability 플랫폼입니다.
- 모든 입력을 tracing하여 입출력 확인, 토큰 사용량 및 비용 확인, 에러 로그 시각화, 지연시간 측정 등의 기능을 제공합니다.
- 아래 세 가지 환경변수만 잘 등록해놓는다면, 따로 코드로 구현하지 않아도 Agent 실행에 대한 모든 로그를 langsmith에서 확인할 수 있습니다.
  - LANGSMITH_TRACING
  - LANGSMITH_API_KEY
  - LANGSMITH_PROJECT

## 2. 단일 Model 호출

- Agent 없이 단일 Model을 호출하는 방법을 알아봅니다.
- invoke()는 모델 또는 체인을 한 번 실행하여 모델의 응답을 반환받는 메서드입니다.
- Agent와 체인을 만들기 전에, 단일 Model을 호출해봅시다.

In [2]:
# OpenAI 채팅 모델 초기화의 가장 기본적인 형태
from langchain.chat_models import init_chat_model
model = init_chat_model("openai:gpt-5-nano") # api key는 환경 변수에서 자동으로 로드됩니다. api_key 매개변수로 직접 전달할 수도 있습니다.

In [4]:
# invoke 메서드로 독립적인 LLM API를 호출합니다.
response = model.invoke('안녕')

# 모델이 보내준 응답 전체를 출력하면, 토큰 사용량 등 메타 데이터가 함께 출력됩니다.
pprint(response)

AIMessage(content='안녕하세요! 무엇을 도와드릴까요? 궁금한 점 풀이, 글쓰기나 번역, 코딩 문제, 공부 도움 등 어떤 주제든 말씀해 주세요. 시작하고 싶은 주제나 질문이 있나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 443, 'prompt_tokens': 8, 'total_tokens': 451, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CXmh5CiBdKAbhP2iJBLJ0Rr8bKIi9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--47911597-a386-4e4b-a5e0-5d5cc65c251e-0', usage_metadata={'input_tokens': 8, 'output_tokens': 443, 'total_tokens': 451, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 384}})


In [5]:
# 모델이 보내준 실제 답변 내용만 출력하려면 응답의 content 속성을 참조합니다.
pprint(response.content)

('안녕하세요! 무엇을 도와드릴까요? 궁금한 점 풀이, 글쓰기나 번역, 코딩 문제, 공부 도움 등 어떤 주제든 말씀해 주세요. 시작하고 싶은 '
 '주제나 질문이 있나요?')


In [6]:
# 모델 초기화 시 다양한 파라미터를 지정할 수 있습니다.
# OpenAI 모델 외 다양한 모델에 대해서도 동일한 파라미터로 요청을 보낼 수 있습니다.
model = init_chat_model(
    model = "openai:gpt-5-nano",
    temperature=0.8, # 응답의 창의성. GPT-5 계열 모델들은 temperature 파라미터를 지원하지 않지만, Langchain에서는 일관된 인터페이스를 제공하기 때문에 오류가 발생하지 않습니다.
    max_tokens=2000, # 최대 토큰 수
    timeout=20, # 요청 제한 시간(초)
    max_retries=2, # 요청 실패 시 재시도 횟수
    )

In [7]:
# 파라미터가 적용된 모델로도 다시 호출해봅시다.
response = model.invoke('너 웹 서치 기능도 가지고 있니?') # 기본 모델은 웹 서치 기능을 가지고 있지 않습니다.
pprint(response.content)

('상황에 따라 다릅니다. 기본적으로는 웹 검색이 자동으로 활성화되어 있지 않지만, 사용 중인 플랫폼에서 브라우징 기능을 켜면 실시간으로 웹 '
 '검색이 가능합니다.\n'
 '\n'
 '- 웹 검색이 가능하면: "웹 검색으로 찾아줘" 같은 요청으로 최신 정보를 찾아 드립니다(출처도 함께 제공 가능).\n'
 '- 웹 검색이 불가능하면: 제 지식 범위(주로 2024년 6월까지) 내에서 답변하고, 필요한 경우 일반적인 방법이나 출처 확인 방법을 '
 '안내해 드립니다.\n'
 '\n'
 '원하시면 현재 세션에서 웹 검색을 사용할 수 있는지 확인해 드릴까요? 그리고 찾고 싶은 주제나 키워드를 알려주시면 그에 맞춰 '
 '진행하겠습니다.')


## 3. Agent 생성

- 아무 추가 기능 없는 단일 모델 호출하려고 Langchain을 쓰는 건 아니죠.
- Langchain의 핵심 기능인 Agent를 만들어보겠습니다.

In [3]:
# Agent 생성에 필요한 모듈입니다.
from langchain.agents import create_agent

# 앞서 초기화한 Model을 사용하여 Agent를 생성합니다.
agent = create_agent(
    model=model, # 사용할 LLM 모델을 미리 초기화(init)한 후 인자로 전달하세요.
    tools=[] # tool은 필수 인자지만, 빈 리스트를 전달하면 아무 tool도 없는 Agent를 생성할 수 있습니다.
    )

In [9]:
# 모델에 전달할 메시지를 OpenAI의 Chat API 형식에 맞게 작성합니다.
# 다른 모델은 다른 형식을 요구할 수 있으니, 모델 문서를 참고하세요.
messages = [
    {"role": "user", "content": "오늘 서울 날씨는 어때?"},
]

# Agent를 통해 모델을 호출합니다. Agent에 대한 invoke는 모델 invoke와는 다르게 dictionary 형태의 입력을 받으니 주의하세요.
# 검색 tool이 없기 때문에, 실시간 날씨 정보를 제공하지는 못할 것입니다.
response = agent.invoke({"messages":messages})
pprint(response)

{'messages': [HumanMessage(content='오늘 서울 날씨는 어때?', additional_kwargs={}, response_metadata={}, id='68ed3ca3-a1c0-4b2c-9d8b-5861c6b824b0'),
              AIMessage(content='지금은 실시간 날씨 정보를 직접 조회할 수 있는 상태가 아니라 오늘 서울 날씨를 바로 알려드리진 못해요. 원하시면 인터넷에서 현재 날씨를 찾아 알려드릴게요.\n\n검색해 드릴까요? 원하신다면 다음 정보를 같이 알려드리겠습니다.\n- 현재 기온\n- 강수확률\n- 강수 유형(비/눈/빗방울)\n- 바람 세기/ 방향\n\n또는 간단한 팁이 필요하시면 오늘의 일반적인 외출 팁도 드릴 수 있어요. 어떤 방식으로 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1101, 'prompt_tokens': 14, 'total_tokens': 1115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 960, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CXmhNfbxQjFMRlAyvcAlAa2zcmfqx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--c486a49b-dc18-

In [10]:
# 실제 답변 내용만 출력하려면 messages 리스트의 마지막 요소의 content 속성을 참조합니다.
pprint(response["messages"][-1].content)

('지금은 실시간 날씨 정보를 직접 조회할 수 있는 상태가 아니라 오늘 서울 날씨를 바로 알려드리진 못해요. 원하시면 인터넷에서 현재 날씨를 '
 '찾아 알려드릴게요.\n'
 '\n'
 '검색해 드릴까요? 원하신다면 다음 정보를 같이 알려드리겠습니다.\n'
 '- 현재 기온\n'
 '- 강수확률\n'
 '- 강수 유형(비/눈/빗방울)\n'
 '- 바람 세기/ 방향\n'
 '\n'
 '또는 간단한 팁이 필요하시면 오늘의 일반적인 외출 팁도 드릴 수 있어요. 어떤 방식으로 도와드릴까요?')


In [11]:
# Agent 생성 시에도 다양한 파라미터를 지정할 수 있습니다.
# 오늘 다 다뤄보지는 못하겠지만, Agent의 세부 동작은 모두 이 파라미터들로 조정할 수 있습니다.
# 아래 있는 것들 외에도 더 많은 파라미터가 있으니 공식 문서를 참고하세요.
agent = create_agent(
    model=model,
    tools=[], # tool 리스트는 다음 챕터에서 다뤄보겠습니다.
    system_prompt="You are a helpful assistant.", # Agent의 시스템 프롬프트입니다.
    middleware=[], # Agent의 동작 사이에 발생하는 요청이나 응답을 가로채서 처리하는 고급 기능입니다.
)

## 4. tool 정의

- 위에서 만든 Agent는 별다른 기능이 없으니 단일 모델 호출과 별로 다를 게 없죠.
- 이제 Agent가 사용할 tool을 만들어 봅시다.
- tool의 종류를 세 가지로 나누어 각각 만들어보겠습니다.

### 4-1. 커스텀 tool 정의

- 직접 만든 Python 함수를 tool로 사용할 수 있습니다.
- URL 또는 라이브러리를 통해 API를 호출하여 응답을 반환하는 tool도 이 방법으로 구현할 수 있습니다.

In [4]:
# tool 정의를 위해 필요한 모듈입니다.
from langchain.tools import tool

# 커스텀 tool 생성
@tool # 데코레이터를 사용하여 tool로 등록합니다.
def calculator(num_1:int, num_2:int) -> int: # typehint는 Agent가 tool의 입출력 형식을 이해하는 데 도움을 줍니다. 안정적인 작동을 위해 반드시 작성하는게 좋습니다.
    """입력받은 두 수의 덧셈을 반환합니다.""" # docstring은 tool의 설명으로 사용됩니다. Agent가 tool을 선택하는 데 도움을 줍니다.
    return num_1 + num_2

### 4-2. langchain-community에 통합된 외부 tool 사용

- 라이브러리를 통해 제공되는 외부 tool들을 langchain-community에서 통합하여 사용할 수 있습니다.
- 웬만한 기능들은 langchain 생태계 내에서 쉽게 쓸 수 있으니 목록을 확인하고 활용해봅시다.
- https://docs.langchain.com/oss/python/integrations/providers/overview

In [5]:
# langchain community에서 통합되어 있는 외부 tool의 목록을 확인해봅시다.
import importlib, pkgutil

package = importlib.import_module("langchain_community.tools")

for mod in pkgutil.iter_modules(package.__path__):
    print(mod.name)

ainetwork
amadeus
arxiv
asknews
audio
azure_ai_services
azure_cognitive_services
bearly
bing_search
brave_search
cassandra_database
clickup
cogniswitch
connery
convert_to_openai
databricks
dataforseo_api_search
dataherald
ddg_search
e2b_data_analysis
edenai
eleven_labs
few_shot
file_management
financial_datasets
github
gitlab
gmail
golden_query
google_books
google_cloud
google_finance
google_jobs
google_lens
google_places
google_scholar
google_search
google_serper
google_trends
graphql
human
ifttt
interaction
jina_search
jira
json
memorize
merriam_webster
metaphor_search
mojeek_search
multion
nasa
nuclia
office365
openai_dalle_image_generation
openapi
openweathermap
passio_nutrition_ai
playwright
plugin
polygon
powerbi
pubmed
render
requests
riza
scenexplain
searchapi
searx_search
semanticscholar
shell
slack
sleep
spark_sql
sql_database
stackexchange
steam
steamship_image_generation
tavily_search
vectorstore
wikidata
wikipedia
wolfram_alpha
yahoo_finance_news
you
youtube
zapier
zenguar

In [6]:
# duckduckgo는 api key 없이도 사용할 수 있는 웹 검색 도구입니다.
import langchain_community.tools.ddg_search

# duckduckgo 검색 tool의 속성을 확인하여 검색 기능 호출 방법을 알아봅시다.
print(dir(langchain_community.tools.ddg_search))

['DuckDuckGoSearchRun', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'tool']


In [7]:
# 위에서 확인한 duckduckgo의 검색 기능을 import합니다.
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun

# 검색 tool 인스턴스를 생성합니다.
ddg_search_tool = DuckDuckGoSearchRun()

In [19]:
# 세 번째 방법을 배우기 전에, 위에서 만든 두 가지 tool을 Agent에게 전달해봅시다.
# nano 모델은 Agent로 사용하기엔 너무 성능이 낮아 무한 루프 등 의도하지 않은 동작이 발생할 수 있으므로 mini 모델로 변경하겠습니다.
model = init_chat_model("openai:gpt-5-mini")

# agent 생성 및 tool 전달
agent = create_agent(
    model=model,
    tools=[calculator, ddg_search_tool], # 앞서 만든 두 가지 tool을 리스트로 전달합니다.
)

In [20]:
# Agent에 전달할 메시지 객체를 생성하기 위해 필요한 모듈입니다.
from langchain.messages import HumanMessage

# Agent에 전달할 메시지 객체를 생성합니다.
messages = [
    HumanMessage(content="2 더하기 3은 얼마야? 그리고 오늘 서울 날씨는 어때?"),
]

# Agent에 대한 invoke는 모델 invoke와는 다르게 dictionary 형태의 입력을 받으니 주의하세요.
response = agent.invoke({"messages":messages})


# Agent가 보내준 전체 응답에는 추론 과정과 tool 사용 내역이 포함되어 있습니다.
pprint(response)

{'messages': [HumanMessage(content='2 더하기 3은 얼마야? 그리고 오늘 서울 날씨는 어때?', additional_kwargs={}, response_metadata={}, id='4e50331f-c479-43ee-9ceb-8dd97768e2fc'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 259, 'prompt_tokens': 207, 'total_tokens': 466, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CXmlh5PQsDctauxKmSLuiaGu1yzA4', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--42b421b0-6a7e-4a04-b129-f70b6a658e77-0', tool_calls=[{'name': 'calculator', 'args': {'num_1': 2, 'num_2': 3}, 'id': 'call_nSfkHuKvnxblNurVOoL8cOs0', 'type': 'tool_call'}, {'name': 'duckduckgo_search', 'args': {'query': '오늘 서울

In [21]:
# 실제 답변 내용만 출력하려면 messages 리스트의 마지막 요소의 content 속성을 참조합니다.
pprint(response["messages"][-1].content)

('2 더하기 3 = 5.\n'
 '\n'
 '오늘 서울 날씨는 지금 바로 실시간으로 확인해 드릴까요? (현재 기온·체감·습도·강수확률·풍속·미세먼지 중 어떤 정보를 원하시는지도 '
 '알려주시면 더 정확히 알려드릴게요.)')


### 4-3. 모델 vendor들이 제공하는 tool을 모델에 바인딩

#### **※ 주의!! ※**
- 현재 버전에서 Agent를 통하지 않고 단일 Model을 직접 호출해야만 사용할 수 있는 기능입니다.
- Agent를 활용하는 경우 사용할 수 없는 방법이므로 헷갈리지 마세요!
- 설계가 복잡해지고 세밀한 구조 설계, 유지보수를 어렵게 만드는 방식이니 사용하지 마세요.
- 다만, 이 방법을 아예 모르고 있으면 GPT나 Docs 보고 코드 쓰다가 tool 정의 및 호출 방법을 섞어서 쓰다가 코드가 꼬일 수 있기 때문에 개념만 이해하고 넘어갑시다.
---
#### 실습
- LLM API를 제공하는 vendor들은 모델에서 곧바로 이용할 수 있는 tool을 함께 서비스하고 있습니다.
- 예를 들어, OpenAI의 경우 웹 서치, MCP 서버 연결, 벡터 스토어 검색, 코드 실행 등의 도구를 제공합니다.
- vendor가 제공하는 tool을 Agent에 바인딩하여 사용해보겠습니다.

In [22]:
from langchain_openai import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0.5,
    model_name="gpt-5-mini",  # 모델명
)

# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
print(f"[답변]: {llm.invoke(question)}")

[답변]: content='대한민국의 수도는 서울특별시(서울)입니다.  \n참고로 정부의 일부 행정 기능과 부처는 세종특별자치시로 이전되어 있어 주요 행정업무는 세종에도 분산되어 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 190, 'prompt_tokens': 15, 'total_tokens': 205, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CXmmMxNr6v6Rh4HXnqWiUYuSCN7GC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--66750ee1-9b9c-4cee-9387-cb3c3d04b1d1-0' usage_metadata={'input_tokens': 15, 'output_tokens': 190, 'total_tokens': 205, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 128}}


In [24]:
# 모델 vendor들이 제공하는 tool을 모델에 바인딩하기 위해 필요한 모듈입니다.
from langchain_openai import ChatOpenAI

# OpenAI 전용 Model 초기화 메서드입니다.
# OpenAI 모델에서 제공하는 tool을 바인딩하기 위해서는 모델 초기화시 ChatOpenAI 클래스를 사용해야 합니다.
model = ChatOpenAI(
    model="gpt-5-mini",
)

# ChatOpenAI 모델 인스턴스에 커스텀 tool을 바인딩합니다.


# 모델이 사용할 tool 리스트를 정의합니다.
# 딕셔너리 형태의 web search tool은 openai가 제공하는 tool이고, calculator는 위에서 직접 만든 커스텀 tool입니다.
# ddg는 웹 검색 역할이 중복되므로 제외합니다.
tools = [{"type": "web_search"}, calculator]

model_with_calculator = model.bind_tools(tools) 

# model을 호출합니다.
response = model_with_calculator.invoke(
    "올해 11월에 한국에서 개봉하는 주요 영화 3개만 알려줘. 그리고 12311 더하기 112455는 몇이야?",
)

# 추론 과정에서 커스텀 tool이 호출되면, 추론 루프를 멈추고 AIMessage를 반환합니다.
# Agent가 아닌 단일 모델에 대한 invoke이기 때문에 커스텀 tool 호출에는 대응하지 못하기 때문입니다.
# 별도로 다시 invoke를 체이닝하는 등 추가 코드가 필요하나, 너무 구린 방식이니까 그냥 넘어가겠습니다.
pprint(response)

AIMessage(content=[{'id': 'rs_0ecc7f32db3c8f7a00690ad44e322c819186ef07e9249ee1b3', 'summary': [], 'type': 'reasoning'}, {'id': 'ws_0ecc7f32db3c8f7a00690ad4536b4c81919be079aec1736eef', 'action': {'query': '2025년 11월 한국 개봉 영화 주요작', 'type': 'search'}, 'status': 'completed', 'type': 'web_search_call'}, {'id': 'rs_0ecc7f32db3c8f7a00690ad4555eb88191b07ccf1d55632dad', 'summary': [], 'type': 'reasoning'}, {'id': 'ws_0ecc7f32db3c8f7a00690ad45822288191b8b77d484b82de7f', 'action': {'query': '위키드 포 굿 한국 개봉 2025 11월 19 CGV', 'type': 'search'}, 'status': 'completed', 'type': 'web_search_call'}, {'id': 'rs_0ecc7f32db3c8f7a00690ad459bb9c8191977cd36c280a2e74', 'summary': [], 'type': 'reasoning'}, {'id': 'ws_0ecc7f32db3c8f7a00690ad45cbca081919e81be10ed228cd0', 'action': {'query': "2025년 11월 개봉 영화 목록 '개봉' '11월' site:movie.naver.com", 'type': 'search'}, 'status': 'completed', 'type': 'web_search_call'}, {'id': 'rs_0ecc7f32db3c8f7a00690ad45efa548191950ef8e1e7ae38ab', 'summary': [], 'type': 'reasoning'}, {'

## 5. 메모리

- 맥락을 유지하고 개인화된 응답을 제공하기 위해 checkpointer 객체에 메모리를 저장합니다.
- 스레드 내에서만 유지할 메모리와 DB에 저장할 메모리를 구분하여 메모리 객체를 생성해보겠습니다.

#### 5-1. 단기 메모리

- 세션이 유지되는 동안 대화 기록을 저장해보겠습니다.
- 세션이 종료되면 이 기억은 사라지며, 복구할 수 없습니다.

In [8]:
# 쓰레드 내 단기 기억 저장용 객체를 생성하기 위한 모듈입니다.
from langgraph.checkpoint.memory import InMemorySaver

# InMemorySaver 객체를 포함하여 Agent 생성
agent = create_agent(
    model=model,
    tools=[calculator, ddg_search_tool], # 앞서 만든 두 가지 tool을 리스트로 전달합니다.
    checkpointer=InMemorySaver(),
)

In [11]:
# Agent에 전달할 메시지 객체를 생성하기 위해 필요한 모듈입니다.
from langchain.messages import HumanMessage

# Agent에 전달할 메시지 객체를 생성합니다.
messages = [
    HumanMessage(content="한국에서 가볼 만한 여행지 추천해줘."),
]

In [ ]:
# Agent 호출
# Agent에 대한 invoke는 모델 invoke와는 다르게 dictionary 형태의 입력을 받으니 주의하세요.
response = agent.invoke(
    {"messages":messages},
    # 설정값으로 대화 기록을 저장할 쓰레드 번호를 함께 넘겨줘야 합니다.
    # 대화 기록이 들어 있는 쓰레드는 세션 종료시 삭제됩니다.
    {"configurable": {"thread_id": "1"}},
    )

pprint(response)

{'messages': [HumanMessage(content='한국에서 가볼 만한 여행지 추천해줘.', additional_kwargs={}, response_metadata={}, id='f04f6b8f-4617-42ed-8daf-cd85fbbb3eba'),
              AIMessage(content='좋아요! 한국에서 가볼 만한 여행지, berbagai 분위기로 골라봤어요. 계절이나 취향에 맞춰 더 구체적인 일정도 도와드릴게요.\n\n- 제주도\n  - 매력: 한라산, 성산일출봉, 오름 트레킹, 제주 올레길, 푸른 바다와 unique 해산물\n  - 좋은 시기: 봄/가을\n  - 팁: 비행기로 1.5시간 내외, 3~4일 추천\n\n- 부산\n  - 매력: 해운대·광안대교 해변, 감천문화마을, 자갈치 시장, 해동용궁사\n  - 좋은 시기: 봄/가을(바다 돋보임)\n  - 팁: KTX로 서울-부산 2.5~3시간\n\n- 경주\n  - 매력: 불국사·석굴암 같은 유네스코 문화유산, 안압지, 양동마을\n  - 좋은 시기: 봄/가을\n  - 팁: 역사 코스로 하루 혹은 이틀 충분\n\n- 순천만 + 보성 녹차밭\n  - 매력: 순천만 습지의 철새, 보성의 녹차밭 드라이브\n  - 좋은 시기: 봄/가을\n  - 팁: 순천은 1~2일, 보성은 차밭 체험 추가\n\n- 여수\n  - 매력: 이순신 광장, 오동도, 해상 케이블카, 맛있는 해산물\n  - 좋은 시기: 봄/여름\n  - 팁: 항구 도시의 야경과 바다 풍경이 좋음\n\n- 전주 한옥마을\n  - 매력: 한옥 체험, 전통 한식(비빔밥), 전통문화\n  - 좋은 시기: 봄/가을\n  - 팁: 도시 중심에 숙소 잡고 도보로 탐방 추천\n\n- 강릉/경포대 및 커피거리\n  - 매력: 경포 해변, 정동진, 커피 문화\n  - 좋은 시기: 여름(바다) 또는 가을(단풍)\n  - 팁: 기차/고속버스 이동 용이\n\n- 설악산(속초)\n  - 매력: 아름다운 단풍과 암봉, 케이블카로 편하게 전망\n  - 좋

In [14]:
# 실제 답변 내용만 출력하려면 messages 리스트의 마지막 요소의 content 속성을 참조합니다.
pprint(response["messages"][-1].content)

('좋아요! 한국에서 가볼 만한 여행지, berbagai 분위기로 골라봤어요. 계절이나 취향에 맞춰 더 구체적인 일정도 도와드릴게요.\n'
 '\n'
 '- 제주도\n'
 '  - 매력: 한라산, 성산일출봉, 오름 트레킹, 제주 올레길, 푸른 바다와 unique 해산물\n'
 '  - 좋은 시기: 봄/가을\n'
 '  - 팁: 비행기로 1.5시간 내외, 3~4일 추천\n'
 '\n'
 '- 부산\n'
 '  - 매력: 해운대·광안대교 해변, 감천문화마을, 자갈치 시장, 해동용궁사\n'
 '  - 좋은 시기: 봄/가을(바다 돋보임)\n'
 '  - 팁: KTX로 서울-부산 2.5~3시간\n'
 '\n'
 '- 경주\n'
 '  - 매력: 불국사·석굴암 같은 유네스코 문화유산, 안압지, 양동마을\n'
 '  - 좋은 시기: 봄/가을\n'
 '  - 팁: 역사 코스로 하루 혹은 이틀 충분\n'
 '\n'
 '- 순천만 + 보성 녹차밭\n'
 '  - 매력: 순천만 습지의 철새, 보성의 녹차밭 드라이브\n'
 '  - 좋은 시기: 봄/가을\n'
 '  - 팁: 순천은 1~2일, 보성은 차밭 체험 추가\n'
 '\n'
 '- 여수\n'
 '  - 매력: 이순신 광장, 오동도, 해상 케이블카, 맛있는 해산물\n'
 '  - 좋은 시기: 봄/여름\n'
 '  - 팁: 항구 도시의 야경과 바다 풍경이 좋음\n'
 '\n'
 '- 전주 한옥마을\n'
 '  - 매력: 한옥 체험, 전통 한식(비빔밥), 전통문화\n'
 '  - 좋은 시기: 봄/가을\n'
 '  - 팁: 도시 중심에 숙소 잡고 도보로 탐방 추천\n'
 '\n'
 '- 강릉/경포대 및 커피거리\n'
 '  - 매력: 경포 해변, 정동진, 커피 문화\n'
 '  - 좋은 시기: 여름(바다) 또는 가을(단풍)\n'
 '  - 팁: 기차/고속버스 이동 용이\n'
 '\n'
 '- 설악산(속초)\n'
 '  - 매력: 아름다운 단풍과 암봉, 케이블카로 편하게 전망\n'
 '  - 좋은 시기: 가을\n'
 '  - 팁: 국

In [15]:
# LLM이 이전 대화 맥락을 잘 기억하는지 확인해봅시다.

# Agent에 전달할 메시지 객체를 생성합니다.
messages = [
    HumanMessage(content="서울 안에서는?"),
]

response = agent.invoke(
    {"messages":messages},
    # 설정값으로 대화 기록을 저장할 쓰레드 번호를 함께 넘겨줘야 합니다.
    # 대화 기록이 들어 있는 쓰레드는 세션 종료시 삭제됩니다.
    {"configurable": {"thread_id": "1"}},
    )

pprint(response)

{'messages': [HumanMessage(content='한국에서 가볼 만한 여행지 추천해줘.', additional_kwargs={}, response_metadata={}, id='f04f6b8f-4617-42ed-8daf-cd85fbbb3eba'),
              AIMessage(content='좋아요! 한국에서 가볼 만한 여행지, berbagai 분위기로 골라봤어요. 계절이나 취향에 맞춰 더 구체적인 일정도 도와드릴게요.\n\n- 제주도\n  - 매력: 한라산, 성산일출봉, 오름 트레킹, 제주 올레길, 푸른 바다와 unique 해산물\n  - 좋은 시기: 봄/가을\n  - 팁: 비행기로 1.5시간 내외, 3~4일 추천\n\n- 부산\n  - 매력: 해운대·광안대교 해변, 감천문화마을, 자갈치 시장, 해동용궁사\n  - 좋은 시기: 봄/가을(바다 돋보임)\n  - 팁: KTX로 서울-부산 2.5~3시간\n\n- 경주\n  - 매력: 불국사·석굴암 같은 유네스코 문화유산, 안압지, 양동마을\n  - 좋은 시기: 봄/가을\n  - 팁: 역사 코스로 하루 혹은 이틀 충분\n\n- 순천만 + 보성 녹차밭\n  - 매력: 순천만 습지의 철새, 보성의 녹차밭 드라이브\n  - 좋은 시기: 봄/가을\n  - 팁: 순천은 1~2일, 보성은 차밭 체험 추가\n\n- 여수\n  - 매력: 이순신 광장, 오동도, 해상 케이블카, 맛있는 해산물\n  - 좋은 시기: 봄/여름\n  - 팁: 항구 도시의 야경과 바다 풍경이 좋음\n\n- 전주 한옥마을\n  - 매력: 한옥 체험, 전통 한식(비빔밥), 전통문화\n  - 좋은 시기: 봄/가을\n  - 팁: 도시 중심에 숙소 잡고 도보로 탐방 추천\n\n- 강릉/경포대 및 커피거리\n  - 매력: 경포 해변, 정동진, 커피 문화\n  - 좋은 시기: 여름(바다) 또는 가을(단풍)\n  - 팁: 기차/고속버스 이동 용이\n\n- 설악산(속초)\n  - 매력: 아름다운 단풍과 암봉, 케이블카로 편하게 전망\n  - 좋

In [16]:
# 실제 답변 내용만 출력하려면 messages 리스트의 마지막 요소의 content 속성을 참조합니다.
pprint(response["messages"][-1].content)

('좋아요. 서울 안에서도 충분히 다양한 분위기의 코스를 즐길 수 있어요. 취향이나 일정에 맞춰 더 구체적으로 맞춤도 가능하니, 원하시는 '
 '방향을 알려주시면 바로 짜드릴게요.\n'
 '\n'
 '추천 코스(테마별)\n'
 '\n'
 '1) 도심 문화+전통 체험 루트\n'
 '- 경복궁 → 덕수궁 산책 → 청계천 산책로 걷기 → 북촌 한옥마을과 삼청동 카페/거리 산책\n'
 '- 포인트: 궁궐 입장 시간대를 살려 오전에 시작하면 덜 붐-focus. 광화문, 경회루 주변 사진 포인트 많음.\n'
 '\n'
 '2) 한강과 자연 힐링 루트\n'
 '- 여의도 한강공원 산책/자전거 타기 → 반포대교 달빛분수 감상(야간 추천) → 서울숲이나 반포 석촌호수 공원까지 연결\n'
 '- 포인트: 도심 속 피크닉 느낌, 계절별로 바람·물소리 즐김. 날씨 good일 때 특히 좋습니다.\n'
 '\n'
 '3) 핫플 카페·맛집 탐방 루트\n'
 '- 홍대/연남동 카페 골목 + 망원동/가로수길 맛집 탐방\n'
 '- 포인트: 트렌디한 분위기, 사진 찍기 좋은 골목길 많음. 걷는 양이 많으니 편한 복장 추천.\n'
 '\n'
 '4) 쇼핑·야간 번화가 루트\n'
 '- 명동/을지로(골목 음식) → 동대문 디자인 플라자(DDP) 야경 → 남대문시장 야시장\n'
 '- 포인트: 쇼핑과 먹거리 집중. 야간에도 활기찬 분위기.\n'
 '\n'
 '5) 가족·데이트 추천 루트\n'
 '- 코엑스몰/코엑스 아쿠아리움 → 봉은사 → 석촌호수 산책\n'
 '- 포인트: 실내-실외를 오가며 여유로운 일정 구성 가능. 아이들에게도 좋음.\n'
 '\n'
 '실내 대안(날씨 안 좋거나 실내 위주 원하실 때)\n'
 '- 코엑스 아쿠아리움, 롯데월드몰/롯데월드타워, 서울시립미술관/SeMA, War Memorial of Korea(전쟁기념관)\n'
 '- 포인트: 비나 눈이 와도 알찬 하루를 보낼 수 있음.\n'
 '\n'
 '간단한 하루 코스 예시\n'
 '- 도심 문화 코스: 경복궁(오전) → 북

### 5-2. 장기 메모리

- 위에서 InMemorySaver로 만든 기억은 세션이 사라지면 함께 없어진다고 했습니다.
- 사용자가 앱을 껐다 켜고, 기존에 대화를 나눴던 세션에 들어왔을 때도 기억이 남아있으려면, 대화 기록을 영속적으로 저장해야 합니다.
- sqlite DB에 대화 기록을 저장할 수 있는 Agent를 만들어보겠습니다.
- 다른 DB를 사용할 경우 해당 DB 관련 패키지 install 및 import가 필요합니다.

In [ ]:
# sqlite용 checkpointer 객체 생성을 위한 모듈입니다.
from langgraph.checkpoint.sqlite import SqliteSaver

# checkpointer 객체 생성
checkpointer = SqliteSaver.from_conn_string("checkpoints.db")

# DB는 with문 안에서만 연결되어 있습니다.
with SqliteSaver.from_conn_string("checkpoints.db") as checkpointer:
    # 테이블이 없을 경우 생성
    checkpointer.setup()
    # Agent 객체 생성
    agent = create_agent(
        model=model,
        tools=tools,
        checkpointer=checkpointer,
    )
    # 실행 후, sqlite DB에 대화 기록이 잘 저장되었는지 확인해보세요.
    # Agent에 전달할 메시지 객체를 생성합니다.
    messages = [
        HumanMessage(content="안녕?"),
    ]

    # Agent invoke
    response = agent.invoke(
        {"messages":messages},
        {"configurable": {"thread_id":1}},
        )

    # 출력
    pprint(response)


{'messages': [HumanMessage(content='서울 안에서는?', additional_kwargs={}, response_metadata={}, id='56162ef8-f25f-4675-932a-9bf25b330738'),
              AIMessage(content=[{'id': 'rs_0fc6438b81b8efa700690ad71452d8819e8ba8111b5c2caa0c', 'summary': [], 'type': 'reasoning'}, {'type': 'text', 'text': '무엇에 대해 여쭤보신 건가요—좀 더 구체적으로 알려주실 수 있나요?\n\n가능한 의미 예시:\n- 서울 안에서는 어디를 가볼까요? (관광·맛집 추천)\n- 서울 안에서는 대중교통/택시 이용이 어떤가요?\n- 서울 안에서는 배달/택배가 되나요?\n- 서울 안에서는 코로나/방역 규칙은 어떤가요?\n- 서울 안에서는 집값·월세는 어느 정도인가요?\n- 서울 안에서는 주차 규정(혹은 자전거 관련) 어떻게 되나요?\n\n원하시는 항목이나 상세 조건(여행일정, 예산, 출발지 등)을 알려주시면 바로 도와드릴게요.', 'annotations': [], 'id': 'msg_0fc6438b81b8efa700690ad719012c819eabfeea13ab32813c'}], additional_kwargs={}, response_metadata={'id': 'resp_0fc6438b81b8efa700690ad713b084819e89a7449b838045ee', 'created_at': 1762318099.0, 'metadata': {}, 'model': 'gpt-5-mini-2025-08-07', 'object': 'response', 'service_tier': 'default', 'status': 'completed', 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07'}, id='resp_0f

In [ ]:
# 저장된 데이터를 조회해보겠습니다.
from langgraph.checkpoint.sqlite import SqliteSaver

# DB를 조회할 때도, with문 아래에서 checkpointer를 호출해야 합니다.
with SqliteSaver.from_conn_string("checkpoints.db") as saver:
    checkpoint_tuple = saver.get_tuple({"configurable": {"thread_id": "1"}})

    # 이하는 db에 저장된 객체에서 사용자와 AI의 대화 부분만 파싱하여 출력하기 위한 코드입니다.
    # 불필요하게 복잡하고 보기에도 불편하므로, langsmith key를 연결해놓고 거기서 확인하는게 좋습니다.
    checkpoint_data = checkpoint_tuple.checkpoint
    messages = checkpoint_data["channel_values"]["messages"]

    for i, msg in enumerate(messages, 1):
        msg_type = getattr(msg, "type", msg.__class__.__name__)
        print(f"{i:02d}. [{msg_type.upper()}] {msg.content}\n")


01. [HUMAN] 서울 안에서는?

02. [AI] [{'id': 'rs_0fc6438b81b8efa700690ad71452d8819e8ba8111b5c2caa0c', 'summary': [], 'type': 'reasoning'}, {'type': 'text', 'text': '무엇에 대해 여쭤보신 건가요—좀 더 구체적으로 알려주실 수 있나요?\n\n가능한 의미 예시:\n- 서울 안에서는 어디를 가볼까요? (관광·맛집 추천)\n- 서울 안에서는 대중교통/택시 이용이 어떤가요?\n- 서울 안에서는 배달/택배가 되나요?\n- 서울 안에서는 코로나/방역 규칙은 어떤가요?\n- 서울 안에서는 집값·월세는 어느 정도인가요?\n- 서울 안에서는 주차 규정(혹은 자전거 관련) 어떻게 되나요?\n\n원하시는 항목이나 상세 조건(여행일정, 예산, 출발지 등)을 알려주시면 바로 도와드릴게요.', 'annotations': [], 'id': 'msg_0fc6438b81b8efa700690ad719012c819eabfeea13ab32813c'}]

03. [HUMAN] 안녕?

04. [AI] [{'id': 'rs_0fc6438b81b8efa700690ad8f9809c819e8f43be7e22c372cd', 'summary': [], 'type': 'reasoning'}, {'type': 'text', 'text': '안녕하세요! 반가워요 — 무엇을 도와드릴까요? 이전에 쓰신 “서울 안에서는?” 관련해서\n- 관광·맛집 추천\n- 교통(지하철·버스·택시)\n- 집값·월세 정보\n- 배달·편의 서비스\n- 방역·행사·날씨 등\n\n원하시는 항목과 날짜·예산·취향 같은 조건을 알려주시면 바로 도와드릴게요.', 'annotations': [], 'id': 'msg_0fc6438b81b8efa700690ad8fce444819eb74f84e66cc7bd83'}]

05. [HUMAN] 안녕?

06. [AI] [{'id': 'rs_0fc6438b81b8efa700690ad97c341c

### ※ 참고
- 단기 메모리든 장기 메모리든, 모델은 쌓여 있는 모든 대화기록을 input받아 대답합니다.
- 따라서 세션 내 대화가 길어질수록 token 사용량이 급격히 늘어나고, 모델이 한 번에 처리할 수 없는 양이 되면 오류를 발생시킵니다.
- 오래된 메시지를 제거하거나 요약하는 등의 방법으로 제어해야 하는데, 여러 방법이 있으나 이번 수업에서는 생략합니다.
- 서비스 개발시 반드시 공식문서를 참조하여 구현해보시기 바랍니다.
  - https://docs.langchain.com/oss/python/langchain/short-term-memory

## 6. 스트리밍

- Agent의 반복적인 추론으로 인한 지연 시간은 UX를 크게 저하시킵니다.
- 출력을 점진적으로 표시하는 스트리밍 기능을 통해 보완해봅시다.
- OpenAI API는 스트리밍 기능 사용을 위해 기관 인증을 요구하므로 지금은 사용할 수 없습니다.
- Ollama 로컬 LLM을 활용하여 맛만 봅시다!

In [ ]:
from langchain_community.chat_models import ChatOllama

# Ollama 데스크탑 앱을 설치하고, 터미널에서 사용할 모델(gemma3)를 미리 다운받아야 합니다.
# ollama pull gemma3

# 모델 지정
model = ChatOllama(
    model="gemma3",
    streaming=True,
)

# Agent 생성 및 tool 전달
agent = create_agent(
    model=model,
)

# Agent에 전달할 메시지 객체를 생성합니다.
messages = [
    HumanMessage(content="Who is the best football player in history?"),
]

In [82]:
# 스트리밍 출력을 확인하기 위해 내용을 파일에 저장하겠습니다.
with open("stream_output.txt", "w", encoding="utf-8") as f:
    # agent.stream()으로 호출
    for token, metadata in agent.stream(
        {"messages": messages}, # HumanMessage 객체 전달
        stream_mode="messages", # 스트림으로 받을 단위 설정
    ):
        # 노트북 셀 출력
        print(f"node: {metadata['langgraph_node']}")
        print(f"content: {token.content_blocks}")
        
        # 파일에 저장
        f.write(f"node: {metadata['langgraph_node']}, ")
        f.write(f"content: {token.content_blocks}")
        f.write('\n')
        f.flush()

node: model
content: [{'type': 'text', 'text': 'Okay'}]
node: model
content: [{'type': 'text', 'text': ','}]
node: model
content: [{'type': 'text', 'text': ' this'}]
node: model
content: [{'type': 'text', 'text': ' is'}]
node: model
content: [{'type': 'text', 'text': ' a'}]
node: model
content: [{'type': 'text', 'text': ' *'}]
node: model
content: [{'type': 'text', 'text': 'fantastic'}]
node: model
content: [{'type': 'text', 'text': '*'}]
node: model
content: [{'type': 'text', 'text': ' and'}]
node: model
content: [{'type': 'text', 'text': ' hugely'}]
node: model
content: [{'type': 'text', 'text': ' debated'}]
node: model
content: [{'type': 'text', 'text': ' question'}]
node: model
content: [{'type': 'text', 'text': '!'}]
node: model
content: [{'type': 'text', 'text': ' There'}]
node: model
content: [{'type': 'text', 'text': "'"}]
node: model
content: [{'type': 'text', 'text': 's'}]
node: model
content: [{'type': 'text', 'text': ' no'}]
node: model
content: [{'type': 'text', 'text': ' 